In [33]:
import pandas as pd
import numpy as np
import cv2
import os
import h5py
# In this we ll load the images, do pre processing to bring data in form 
# for training
# build the cnn model

In [34]:
import keras
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, Activation

In [35]:
# loading the csv files and reading images according to it
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

#reading train images, we already have saved resized images in train_new
trainX = []
for image in train.ID:
    img = cv2.imread(os.path.join('train_new', image), cv2.IMREAD_COLOR)
    img = np.array(img, dtype = 'float32')
    trainX.append(img)
trainX = np.array(trainX)    #or can use np.stack(trainX, axis = 0)
#trainX.shape
#trainX.reshape((len(train), 32, 32, 3))

#similarly for test images
testX = []
for image in test.ID:
    img = cv2.imread(os.path.join('test_new', image), cv2.IMREAD_COLOR)
    img = np.array(img, dtype = 'float32')
    testX.append(img)
testX = np.array(testX) 

In [36]:
trainX = trainX / 255
testX = testX/ 255 #normalizing for fast processing

In [37]:
# now we will make trainY. We dont have classification for test data
trainY1 = train.Class
# since to_categorical converts into one hot encoding fro numbers,
#we first need to convert young, middle, old into numbers
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
trainY2 = le.fit_transform(trainY1)  #fit_transform returns encoded labels
trainY = keras.utils.to_categorical(trainY2, num_classes = 3)
#trainY1[1] = YOUNG
#trainY2[1] = 2
#trainY[1] = [0.,0., 1.]

In [41]:
# setting hyperparameters
batchSize = 128
model = Sequential()

model.add(Convolution2D(filters = 32, kernel_size = (3, 3), padding = 'same',
                        input_shape = (32, 32, 3), activation = 'relu'))

model.add(Convolution2D(filters = 32, kernel_size = (3, 3), padding = 'same',
                        activation = 'relu'))

model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))

model.add(Convolution2D(filters = 64, kernel_size = (3, 3), padding = 'same',
                        activation = 'relu'))

model.add(Convolution2D(filters = 64, kernel_size = (3, 3), padding = 'same',
                       activation = 'relu'))

model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))

model.add(Convolution2D(filters = 128, kernel_size = (3, 3), padding = 'same',
                       activation = 'relu'))

model.add(Convolution2D(filters = 128, kernel_size = (3, 3), padding = 'same',
                       activation = 'relu'))

model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
            
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

In [42]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])

In [ ]:
for i in range(10): 
    model.fit(trainX, trainY, validation_split = 0.1, batch_size = batchSize, epochs = 40,
              verbose = 1)
    model.save(os.getcwd() + '/saved_model/keras_face_age_model_' + str(i*40 + 40) +'.h5')

Train on 17915 samples, validate on 1991 samples
Epoch 1/40
17915/17915 [==============================] - 197s - loss: 0.0342 - acc: 0.9888 - val_loss: 1.0830 - val_acc: 0.8061
Epoch 2/40
17915/17915 [==============================] - 217s - loss: 0.0388 - acc: 0.9873 - val_loss: 1.1356 - val_acc: 0.8046
Epoch 3/40
17915/17915 [==============================] - 222s - loss: 0.0376 - acc: 0.9871 - val_loss: 1.1009 - val_acc: 0.8011
Epoch 4/40
17915/17915 [==============================] - 221s - loss: 0.0400 - acc: 0.9870 - val_loss: 1.0620 - val_acc: 0.8016
Epoch 5/40
17915/17915 [==============================] - 211s - loss: 0.0400 - acc: 0.9871 - val_loss: 1.1917 - val_acc: 0.7961
Epoch 6/40
17915/17915 [==============================] - 185s - loss: 0.0324 - acc: 0.9891 - val_loss: 1.1857 - val_acc: 0.8051
Epoch 7/40
17915/17915 [==============================] - 184s - loss: 0.0296 - acc: 0.9899 - val_loss: 1.1434 - val_acc: 0.8051
Epoch 8/40
17915/17915 [========================

17915/17915 [==============================] - 168s - loss: 0.0188 - acc: 0.9941 - val_loss: 1.4082 - val_acc: 0.8001
Epoch 24/40
 4864/17915 [=======>......................] - ETA: 119s - loss: 0.0289 - acc: 0.9910

In [44]:
model.save(os.getcwd() + '/saved_model/keras_face_age_model_40.h5')